In [1]:
from more_itertools import sliced
import re
import numpy as np
import pandas as pd
import time


from nltk.tokenize import sent_tokenize
from nltk import *


from IPython.display import clear_output

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import time

import torch
from torchtext import data
import torch.nn as nn

import seaborn as sns

import spacy
import os


import pdfplumber  #used to load the pdf

In [2]:
# Load swedish spacy model
# Must have a new version of spacy, so check what version you have

nlp = spacy.load("C:\\Users\\GTSA - Infinity\\Downloads\\sv_model_upos\\sv_model_upos\\sv_model0\\sv_model_upos")

In [3]:
# Loading the data from the Flashback dataset that contains citat and answers 

flash = pd.read_csv("FLASH_CITAT_CLEAN.csv")
flash

,citat,svar,citat_count,svar_count
0,"hmm, säkert intressant... extremt långt att lä...",du noterade inte att tråden var skapad 2001?,13,8
1,"nerå har läst denna förut, nån som har den i p...",haha hur lyckas du?,14,4
2,"nerå har läst denna förut, nån som har den i p...",okynnesbump (tråden är från 2001) tråden lås...,14,8
3,förlåt om jag gräver upp en extremt gammal trå...,nu blir det lite moderator-ot här men jag måst...,22,26
4,my phone explorer? vilket os är det till? elle...,det går att tanka här: http://www.fjsoft.at/en...,17,33
...,...,...,...,...
550462,gör det saken bättre att man skulle sälja det ...,javisst! blir det ett jävla liv så måste man...,10,54
550463,"när pappa och mamma hitta mina sprutor, så ble...",de tungt. men min kurr kostade nästan 10lax so...,41,19
550464,de tungt. men min kurr kostade nästan 10lax so...,vilken idiot! det är strängligen förbjudet att...,19,25
550465,grabbar har ni inte lärt er att gömma sakerna ...,mina saker ligger i ett låst skåp jag har barn...,63,16


In [4]:
#Testing spacy with swedish

doc = nlp("Jag gillar London och Malmö, men Eslöv är ändå den roligaste staden, där bor nämligen min mamma som heter Maria och Maja Emma")
width = 15
for token in doc:
    print(f"{token.text: <{width}} {token.tag_: <{width}} {token.dep_: <{width}}")
    
for ent in doc.ents:
    print(ent.text, ent.label_)

Jag             PRON            nsubj          
gillar          VERB            ROOT           
London          PROPN           obj            
och             CCONJ           cc             
Malmö           PROPN           conj           
,               PUNCT           punct          
men             CCONJ           cc             
Eslöv           PROPN           nsubj          
är              AUX             cop            
ändå            ADV             advmod         
den             DET             det            
roligaste       ADJ             amod           
staden          NOUN            conj           
,               PUNCT           punct          
där             ADV             advmod         
bor             VERB            acl:relcl      
nämligen        ADV             advmod         
min             PRON            nmod:poss      
mamma           NOUN            nsubj          
som             PRON            nsubj          
heter           VERB            acl:relc

In [5]:

doc = nlp(flash.iloc[1,0] + "Adam Svensson")  #testing adding a person
width = 15
for token in doc:
    print(f"{token.text: <{width}} {token.tag_: <{width}} {token.dep_: <{width}}")
    
for ent in doc.ents:
    print(ent.text, ent.label_)

nerå            ADV             advmod         
har             AUX             aux            
läst            VERB            ROOT           
denna           PRON            obj            
förut           ADV             advmod         
,               PUNCT           punct          
nån             PRON            parataxis      
som             PRON            nsubj          
har             VERB            acl:relcl      
den             PRON            obj            
i               ADP             case           
pdf             ADJ             nmod           
månntro         NOUN            obl            
?               PUNCT           punct          
Adam            PROPN           appos          
Svensson        PROPN           flat:name      
Adam Svensson PRS


PDFplumber test

In [6]:
#This is how you load a page from the pdf

import pdfplumber

with pdfplumber.open("Dom gällande mord i Nydalaparken 180703.pdf") as pdf:
    first_page = pdf.pages[25]
    print(first_page.extract_text())

26
 
   
MALMÖ TINGSRÄTT  DOM (DELDOM)  B 5032-18 
Avdelning 2  2019-05-10   
Enhet 204 B 
 
 
 
 
med undantag för tändsatspartikeln, som tyder på att Mustapha Hassoun deltagit på 
det sätt som angivits. Han har bott på Nydala i hela sitt liv och rör sig i området 
varje dag. Därför inte så konstigt att han kan ha iakttagits av vittnen den här dagen. 
Natten mot den 3 juli har det skett en annan skottlossning i området, någonstans i 
närheten. Killen som blev skjuten då är också en nära vän till Mustapha Hassoun. 
Det är därför han åker till sjukhuset i Malmö. Han blir medtagen till polisen som tar 
hans telefon i beslag. Han släpps vid halv sex-tiden den 3 juli. Från första förhöret i 
denna utredning och fram till att förundersökningen var klar har Mustapha Hassoun 
lämnat i stort sett samma berättelse. Han lämnar polisen halv sex på morgonen och 
åker hem till bostaden där han bor med sin familj. Sover till cirka klockan två, då 
han blir väckt. Han går till falafelstället och äter

In [7]:

#Using the page above to see all the NER 

doc = nlp(first_page.extract_text())
width = 15
for token in doc:
    print(f"{token.text: <{width}} {token.tag_: <{width}} {token.dep_: <{width}}")
    
for ent in doc.ents:
    print(ent.text, ent.label_)

       case           
området         NOUN            obl            

               _SP                            
varje           DET             det            
dag             NOUN            obl            
.               PUNCT           punct          
Därför          ADV             advmod         
inte            PART            advmod         
så              ADV             advmod         
konstigt        ADJ             conj           
att             SCONJ           mark           
han             PRON            nsubj:pass     
kan             AUX             aux            
ha              AUX             aux            
iakttagits      VERB            csubj          
av              ADP             case           
vittnen         NOUN            obl:agent      
den             DET             det            
här             ADV             fixed          
dagen           NOUN            obl            
.               PUNCT           punct          

               _

In [8]:

for ent in doc.ents:
    print(ent.text, ent.label_)

MALMÖ LOC
DELDOM LOC
2019-05-10 TME
Mustapha Hassoun PRS
Nydala LOC
varje dag TME
den här dagen TME
den 3 juli TME
Mustapha Hassoun PRS
Malmö LOC
vid halv sex-tiden den 3 juli TME
Mustapha Hassoun PRS
halv sex på morgonen TME
klockan två TME
Mustapha Hassoun PRS
Mustapha Hassoun PRS
Nydala LOC
Mahmoud Shakir PRS
Nydala LOC


In [9]:
# Collect the above output but in lists 
text_lista = []
label_lista = []
for ent in doc.ents:
    text_lista.append(ent.text)
    label_lista.append(ent.label_)
print(len(text_lista))
print(len(label_lista))

19
19


In [10]:
# Samlar all text från en 

with pdfplumber.open("Dom gällande mord i Nydalaparken 180703.pdf") as pdf:
    all_text = ""
    for saker in pdf.pages:
        test = saker.extract_text()
        all_text += str(test)
all_text = " ".join(all_text.split())  #tar bort alla \n från all_text
print(all_text[6000:7000])

alil c/o Advokat Douglas Norking AB Malmöhusvägen 1 211 18 Malmö Målsägandebiträde: Advokat Douglas Norking Advokat Douglas Norking AB Malmöhusvägen 1 211 18 Malmö 2. Gunnar Larme c/o Åklagaren Företrädd av åklagaren 3. Birgit Mikaelsson-Pudjas dödsbo c/o Åklagaren Företrädd av åklagaren6 MALMÖ TINGSRÄTT DOM (DELDOM) Mål nr:B 5032-18 Avdelning 2 2019-05-10 Enhet 204 B 4. Sara Mohammed Shakir c/o Advokat Douglas Norking AB Malmöhusvägen 1 211 18 Malmö Målsägandebiträde: Advokat Douglas Norking Advokat Douglas Norking AB Malmöhusvägen 1 211 18 Malmö 5. Sroud Mohammed Shakir c/o Advokat Anton Strand Advokatfirman Defens AB Kornhamnstorg 57 111 27 Stockholm Målsägandebiträde: Advokat Anton Strand Advokatfirman Defens AB Kornhamnstorg 57 111 27 Stockholm Målsägandebiträde genom substitution: Anna Ekström c/o Advokatfirman Defens Kornhamnstorg 57 111 27 Stockholm 6. Zeynab Mohammed Shakir c/o Advokat Douglas Norking AB Malmöhusvägen 1 211 18 Malmö Målsägandebiträde: Advokat Douglas Norking A

In [11]:
doc = nlp(all_text)


text_lista = []
label_lista = []
for ent in doc.ents:
    text_lista.append(ent.text)
    label_lista.append(ent.label_)
print(len(text_lista))
print(len(label_lista))

1842
1842


In [12]:

df_total = pd.DataFrame(list(zip(text_lista, label_lista)), 
               columns =["Text", "Entitet"]) 

df_total 

,Text,Entitet
0,MALMÖ,LOC
1,DELDOM,ORG
2,Malmö,LOC
3,ZAKRIYA Awad Mezar,PRS
4,Häktad Södra Hunnetorpsvägen,LOC
...,...,...
1837,inom 3 veckor,TME
1838,2 g,MSR
1839,inom 4 veckor,TME
1840,i tid,TME


In [13]:
df = df_total.drop_duplicates()  #tar bort duplicates
df

,Text,Entitet
0,MALMÖ,LOC
1,DELDOM,ORG
2,Malmö,LOC
3,ZAKRIYA Awad Mezar,PRS
4,Häktad Södra Hunnetorpsvägen,LOC
...,...,...
1836,inom 3 veckor,TME
1838,2 g,MSR
1839,inom 4 veckor,TME
1840,i tid,TME


In [14]:
#Granskar vad som har blivit taggade som Locations
df[df["Entitet"] =="LOC"]

,Text,Entitet
0,MALMÖ,LOC
2,Malmö,LOC
4,Häktad Södra Hunnetorpsvägen,LOC
5,Helsingborg,LOC
16,Malmöhusvägen,LOC
38,Stockholm,LOC
82,DELDOM,LOC
120,Lgh,LOC
167,Malmö7,LOC
241,Eriksfältsgatan,LOC


In [15]:
# Granskar vad som har blivit klassificerat som tider

df[df["Entitet"] =="TME"]

,Text,Entitet
23,10 måndag,TME
71,den 3 juli 2018,TME
185,2018-02-21,TME
186,2018-02-23,TME
187,2 tillfällen,TME
...,...,...
1727,på förmiddagen,TME
1813,kl 15.09,TME
1836,inom 3 veckor,TME
1839,inom 4 veckor,TME


In [16]:
#Kollar personer

df[df["Entitet"] =="PRS"]

,Text,Entitet
3,ZAKRIYA Awad Mezar,PRS
6,Lars Kruse,PRS
10,Pär Andersson,PRS
13,Vian,PRS
14,Khalil,PRS
...,...,...
1801,Rinor Dragaj,PRS
1814,Tommas Jönssons,PRS
1815,Joel Magnussons,PRS
1828,2019-03-13 MÅLNR,PRS


In [17]:
nlp = spacy.load("C:\\Users\\GTSA - Infinity\\Downloads\\sv_model_upos\\sv_model_upos\\sv_model0\\sv_model_upos")  #laddar NER modellen
with pdfplumber.open("Dom gällande mord i Nydalaparken 180703.pdf") as pdf:
    all_text = ""
    for saker in pdf.pages:
        test = saker.extract_text()
        all_text += str(test)

    all_text = " ".join(all_text.split())  #tar bort alla \n från all_text
    doc = nlp(all_text)
    text_lista = []
    label_lista = []
    for ent in doc.ents:
        text_lista.append(ent.text)
        label_lista.append(ent.label_)

    df_total = pd.DataFrame(list(zip(text_lista, label_lista)), columns= ["Text", "Entitet"])

    df_total["Text"] = df_total["Text"].str.strip()
    df_total["Text"] = df_total["Text"].str.lower()

    df = df_total.groupby(["Text", "Entitet"]).size().reset_index(name = "counts").sort_values(by = ["counts"], ascending=False)


with pdfplumber.open("Dom gällande mord i Nydalaparken 180703.pdf") as pdf:
    lista = []
    lista_mening = []
    for i in range(len(df)):
        j = 1
        lista_2 = []
        lista_mening_2 = []
        for saker in pdf.pages:
            test = saker.extract_text()
            try:
                all_text = str(test.lower())
                all_text = " ".join(all_text.split())  #tar bort alla \n från all_text
                if df.iloc[i,0] in all_text:
                    lista_2.append(j)
                    j += 1
                    for m in re.finditer(df.iloc[i, 0], all_text):  #hittar ordet och bokstäverna innan och efter
                                texten = all_text[m.start()-40: m.end()+40]
                                if len(texten) < 5:
                                    continue
                                else:
                                    lista_mening_2.append(all_text[m.start()-40: m.end()+40])
                else:
                    j +=1
            except :
                j +=1
        lista.append(lista_2)
        lista_mening.append(lista_mening_2)
        j = 1
df["Pages"] = lista
df["Meningar"] = lista_mening

In [18]:
# Bra sätt att ta bort mellanrum
" ".join(all_text[0:1000].split())

'prövningstillstånd i hovrätten när får man prövningstillstånd? när överklagandet kommer in till hovrätten tar hovrätten ger prövningstillstånd i fyra olika domstolen först ställning till om målet ska tas fall. upp till prövning. \uf0b7 domstolen bedömer att det finns anledning om du inte får prövningstillstånd gäller den att tvivla på att tingsrätten dömt rätt. överklagade domen. därför är det viktigt att i \uf0b7 domstolen anser att det inte går att bedöma överklagandet ta med allt du vill föra fram. om tingsrätten har dömt rätt utan att ta upp målet. när krävs det prövningstillstånd? \uf0b7 domstolen behöver ta upp målet för att ge andra domstolar vägledning i brottmålsdelen rättstillämpningen. i brottmålsdelen behövs prövningstillstånd i \uf0b7 domstolen bedömer att det finns synnerliga två olika fall: skäl att ta upp målet av någon annan anledning. \uf0b7 den åtalade har dömts enbart till böter. \uf0b7 den åtalade har frikänts från ett brott som inte har mer än 6 månaders fängelse

In [19]:
#Här kan man se hur många \n det finns
all_text[0:1000]

'prövningstillstånd i hovrätten när får man prövningstillstånd? när överklagandet kommer in till hovrätten tar hovrätten ger prövningstillstånd i fyra olika domstolen först ställning till om målet ska tas fall. upp till prövning. \uf0b7 domstolen bedömer att det finns anledning om du inte får prövningstillstånd gäller den att tvivla på att tingsrätten dömt rätt. överklagade domen. därför är det viktigt att i \uf0b7 domstolen anser att det inte går att bedöma överklagandet ta med allt du vill föra fram. om tingsrätten har dömt rätt utan att ta upp målet. när krävs det prövningstillstånd? \uf0b7 domstolen behöver ta upp målet för att ge andra domstolar vägledning i brottmålsdelen rättstillämpningen. i brottmålsdelen behövs prövningstillstånd i \uf0b7 domstolen bedömer att det finns synnerliga två olika fall: skäl att ta upp målet av någon annan anledning. \uf0b7 den åtalade har dömts enbart till böter. \uf0b7 den åtalade har frikänts från ett brott som inte har mer än 6 månaders fängelse

In [20]:
df_total

,Text,Entitet
0,malmö,LOC
1,deldom,ORG
2,malmö,LOC
3,zakriya awad mezar,PRS
4,häktad södra hunnetorpsvägen,LOC
...,...,...
1837,inom 3 veckor,TME
1838,2 g,MSR
1839,inom 4 veckor,TME
1840,i tid,TME


In [21]:

df

,Text,Entitet,counts,Pages,Meningar
309,malmö,LOC,178,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",[ing 2 2019-05-10 enhet 204 b meddelad i malmö...
306,mahmoud shakir,PRS,127,"[3, 4, 7, 8, 12, 13, 17, 20, 21, 22, 23, 26, 2...",[i malmö hb stortorget 3 211 22 malmö 6. mahmo...
119,bilal el halabi,PRS,87,"[8, 10, 13, 20, 21, 22, 23, 25, 27, 28, 29, 30...",[ 8. nasim al janabi ska solidariskt med bilal...
339,nasim al janabi,PRS,80,"[5, 8, 13, 19, 20, 21, 22, 24, 25, 28, 29, 31,...",[ enhet 204 b meddelad i malmö tilltalad nasim...
135,deldom,ORG,78,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",[]
...,...,...,...,...,...
184,fem meter,MSR,1,[42],"[e dem såg han tre telefoner på en bänk, fem m..."
183,fadil corbics,PRS,1,[72],[ll 150 000 kr. anledningen att han fick fadil...
180,ett års,TME,1,[70],[ulle bli lurad. för honom är 150 000 kr ett å...
179,ett år tidigare,TME,1,[35],[ar i bilen som han hade gett till denne ett å...


In [32]:
#Kollar de 10 första gångerna som Musti nämns
df.iloc[10,4][0:10]

['rhör berättat att han, nassim, zaki och musti varit på platsen kan det varit för att ',
 'skriven på adjunktsgatan och kallas för musti, soz och sofi. anledningen till att de ',
 ' och hoodie. efter att han beställt kom musti och bilal dit. han hade skrivit till en',
 'stapha, bilal och nasim är hans vänner. musti och bilal var med på falafel n. 1 men h',
 't hotell för att sova där. det var han, musti och nasim. de åkte till sjukhuset för a',
 ' få kontakt med hawita, zaki, bilal och musti. hon ringde dem mellan fem och sex. moh',
 'e som bor precis framför dem kallas för musti, han var med. han känner igen de andra ',
 'te namnen. han har hos polisen pekat ut musti och en till som han inte kan namnet på.',
 't hår. en hade kryckor. han känner igen musti och den som haltar men inte de andra tv',
 'hon ute med hunden. när hon var ute kom musti framför henne och en annan kille. hon h']

In [55]:
# Nämner sturup 
df.iloc[33,4][0:10]

['en stund innan kl 02:00 då de åker till sturup airport hotel. efter att de varit på ho',
 'atsen. han har inte varit på hotellet i sturup eller i arkelstorp. mustapha hassoun mu',
 'nda hotellet som tog emot kontanter var sturup. han fick ett samtal om att poliser kna',
 'a honom. därefter åkte de tillbaka till sturup för att sova. nycklarna som syns på hon',
 'och nasim al janabi lämnat hotellet vid sturups flygplats strax före kl. 13.30 den 3 j',
 'pa med orange tagg synts på bilder från sturup samt att en nyckelknippa med samma utse',
 'hade på sig när han lämnade hotellet på sturup. då hade han också på sig shorts. vittn',
 'zar såg ut på bilderna från hotellet på sturup. allt tyder på att avan amin och salih ',
 'det innan. han har varit på hotellet på sturup under natten till den 3 juli. han har t',
 ' janabi och zakriya mezar sovit över på sturup airport hotel medan bilal el halabi ber']